<a href="https://colab.research.google.com/github/Ravio1i/ki-lab/blob/master/4_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import gym
import random 

!pip3 install box2d-py
import torch
import torch.nn.functional as F
from torch import optim
import numpy as np

from time import time

In [90]:
class Net(torch.nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.fc1(x))
        return F.log_softmax(self.fc2(x))

In [91]:
hidden_dim = 256 #@param {type:"integer"}
n_episodes = 100 #@param {type:"integer"}
n_steps = 500 #@param {type:"integer"}
lr = 1e-3 #@param {type:"number"}
reward_goal = 100 #@param {type:"integer"}

device = torch.device("cuda:0")

In [92]:
env = gym.make("LunarLander-v2")
n_actions = env.action_space.n
n_states = 8 # env.observation_space
print(n_actions)

4


In [93]:
model = Net(
    input_dim = n_states, 
    hidden_dim = hidden_dim, 
    output_dim = n_actions
)
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [114]:
total_reward = 0
state = env.reset()
done = False

def generate_episodes(n_episodes: int, n_steps: int):
    rewards = []
    state_action_reward = []

    for episode in range(1,n_episodes+1):
        state = env.reset()
        state_action_episode = []
        reward_episode = 0
        for s in range(n_steps):
            state = torch.from_numpy(state)
            out = model(state)
            action = torch.argmax(out).item()
            next_state, reward, done, _ = env.step(action)
            state = next_state
            reward_episode += reward
            state_action_episode.append((state, action))
            if done:
                break
        state_action_reward.append((state_action_episode, reward_episode))

        print('\rEpisode {}\tAverage Reward: {:.2f}'.format(episode, np.mean(rewards)), end="")
        rewards.append(reward_episode)
        if episode % 100 == 0:
            print('\rEpisode {}\tAverage Reward: {:.2f}'.format(episode, np.mean(rewards)))

    torch.save(model, 'model.pth')
    return state_action_reward

In [121]:
def train():
    state_action_reward = generate_episodes(n_episodes, n_steps)
    state_action_reward = sorted(state_action_reward, key=lambda t: t[1], reverse=True)[:20]
    print(state_action_reward[0])
    print(len(state_action_reward))

In [120]:
train()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Episode 100	Average Reward: -192.97
20


In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()